In [1]:
import torch
import numpy as np

Creating tensors

In [4]:
np.set_printoptions(precision=3)
a = [1, 2, 3]
b = np.array([4, 5, 6], dtype=np.int32)
t_a = torch.tensor(a)
t_b = torch.from_numpy(b)
print(t_a)
print(t_b)

t_ones = torch.ones(2, 3)
print(t_ones.shape)
print(t_ones)

rand_tensor = torch.rand(2, 3)
print(rand_tensor)

tensor([1, 2, 3])
tensor([4, 5, 6], dtype=torch.int32)
torch.Size([2, 3])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.6643, 0.0147, 0.8987],
        [0.5834, 0.8475, 0.0178]])


Manipulating shape and type of tensor

In [7]:
t_a_new = t_a.to(torch.int64)
print(t_a_new.dtype)

# transpose
t = torch.rand(3, 5)
t_tr = torch.transpose(t, 0, 1)
print(t.shape, ' --> ', t_tr.shape)

# reshape
t = torch.zeros(30)
t_reshape = t.reshape(5, 6)
print(t_reshape.shape)

# remove unecessary dimensions
t = torch.zeros(1, 2, 1, 4, 1)
t_sqz = torch.squeeze(t, 2)     # squeeze dimension 2
print(t.shape, ' --> ', t_sqz.shape)

torch.int64
torch.Size([3, 5])  -->  torch.Size([5, 3])
torch.Size([5, 6])
torch.Size([1, 2, 1, 4, 1])  -->  torch.Size([1, 2, 4, 1])


Mathematical operations

In [9]:
torch.manual_seed(1)
t1 = 2 * torch.rand(5, 2) - 1                       # uniform dist
t2 = torch.normal(mean=0, std=1, size=(5, 2))       # standard normal

t3 = torch.multiply(t1, t2)                         # element-wise product
print(t3)

t4 = torch.mean(t1, axis=0)                         # mean along specified axis
print(t4)

t5 = torch.matmul(t1, torch.transpose(t2, 0, 1))    # matrix-matrix product
print(t5)

t6 = torch.matmul(torch.transpose(t1, 0, 1), t2)    # other matrix product
print(t6)

norm_t1 = torch.linalg.norm(t1, ord=2, dim=1)       # norm
print(norm_t1)

tensor([[ 0.4426, -0.3114],
        [ 0.0660, -0.5970],
        [ 1.1249,  0.0150],
        [ 0.1569,  0.7107],
        [-0.0451, -0.0352]])
tensor([-0.1373,  0.2028])
tensor([[ 0.1312,  0.3860, -0.6267, -1.0096, -0.2943],
        [ 0.1647, -0.5310,  0.2434,  0.8035,  0.1980],
        [-0.3855, -0.4422,  1.1399,  1.5558,  0.4781],
        [ 0.1822, -0.5771,  0.2585,  0.8676,  0.2132],
        [ 0.0330,  0.1084, -0.1692, -0.2771, -0.0804]])
tensor([[ 1.7453,  0.3392],
        [-1.6038, -0.2180]])
tensor([0.6785, 0.5078, 1.1162, 0.5488, 0.1853])


Split, stack and concatenate tensors

In [12]:
torch.manual_seed(1)
# split 
t = torch.rand(6)
print(t)

t_splits = torch.chunk(t, 3)           # number of tensors after split
print([item.numpy() for item in t_splits])


t = torch.rand(5)
print(t)

t_splits = torch.split(t, split_size_or_sections=[3, 2])
print([item.numpy() for item in t_splits])
  

# concatenate
A = torch.ones(3)
B = torch.zeros(2)
C = torch.cat([A, B], axis=0)
print(C)

# stack 
A = torch.ones(3)
B = torch.zeros(3)
S = torch.stack([A, B], axis=1)
print(S)

tensor([0.7576, 0.2793, 0.4031, 0.7347, 0.0293, 0.7999])
[array([0.758, 0.279], dtype=float32), array([0.403, 0.735], dtype=float32), array([0.029, 0.8  ], dtype=float32)]
tensor([0.3971, 0.7544, 0.5695, 0.4388, 0.6387])
[array([0.397, 0.754, 0.57 ], dtype=float32), array([0.439, 0.639], dtype=float32)]
tensor([1., 1., 1., 0., 0.])
tensor([[1., 0.],
        [1., 0.],
        [1., 0.]])


### DataLoader

In [18]:
from torch.utils.data import DataLoader, Dataset

In [16]:
t = torch.arange(6, dtype=torch.float32)
data_loader = DataLoader(t)

for item in data_loader:
    print(item)

data_loader = DataLoader(t, batch_size=3, drop_last=False)
for i, batch in enumerate(data_loader, 1):
    print(f'batch {i}:', batch)

tensor([0.])
tensor([1.])
tensor([2.])
tensor([3.])
tensor([4.])
tensor([5.])
batch 1: tensor([0., 1., 2.])
batch 2: tensor([3., 4., 5.])


Combine tensors into joint dataset

In [19]:
torch.manual_seed(1)
t_x = torch.rand([4, 3], dtype=torch.float32)
t_y = torch.arange(4)

A custom Dataset class must have an init-method, which is where the logic happens (reding existing arrays, loading a file, filtering data) and a getitem-method which returns the corresponding sample for the given index

In [20]:
class JointDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [21]:
joint_dataset = JointDataset(t_x, t_y)

for example in joint_dataset:
    print('  x: ', example[0], ' y: ', example[1])

  x:  tensor([0.7576, 0.2793, 0.4031])  y:  tensor(0)
  x:  tensor([0.7347, 0.0293, 0.7999])  y:  tensor(1)
  x:  tensor([0.3971, 0.7544, 0.5695])  y:  tensor(2)
  x:  tensor([0.4388, 0.6387, 0.5247])  y:  tensor(3)


Shuffling data

In [ ]:
torch.manual_seed(1)
data_loader = DataLoader(dataset=joint_dataset, batch_size=2, shuffle=True)

for i, batch in enumerate(data_loader, 1):
    print(f'batch {i}:', 'x:', batch[0])